In [2]:
import os
import time

import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [3]:
path = 'data'
train_path = os.path.join(path, 'train')
val_path = os.path.join(path, 'val')
test_path = os.path.join(path, 'test')

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_dataset = datasets.ImageFolder(train_path, transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]))
valid_dataset = datasets.ImageFolder(val_path, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))
print(len(train_dataset))

6516


In [6]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    num_workers=4
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=64,
    num_workers=4
)

In [47]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
       
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self. conv1_bn = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)
       
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.conv2_bn = nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3_bn = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3_bn = nn.BatchNorm2d(64)
        
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv4_bn = nn.BatchNorm2d(128)
        
        self.fc1 = nn.Linear(128 * 12 * 12, 1000)
        self.fc2 = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.pool(F.relu(self.conv1_bn(self.conv1(x))))
        x = self.pool(F.relu(self.conv2_bn(self.conv2(x))))
        x = self.pool(F.relu(self.conv3_bn(self.conv3(x))))
        x = self.pool(F.relu(self.conv4_bn(self.conv4(x))))
        x = x.view(-1, 128*12*12)
        x = F.dropout(F.relu(self.fc1(x)), training=self.training, p=0.4)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [10]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5)
        self. conv1_bn = nn.BatchNorm2d(16)
       
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.conv2_bn = nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3_bn = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3_bn = nn.BatchNorm2d(64)
        
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv4_bn = nn.BatchNorm2d(128)
        
        self.fc1 = nn.Linear(128 * 12 * 12, 1000)
        self.fc2 = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.pool(F.relu(self.conv1_bn(self.conv1(x))))
        x = self.pool(F.relu(self.conv2_bn(self.conv2(x))))
        x = self.pool(F.relu(self.conv3_bn(self.conv3(x))))
        x = self.pool(F.relu(self.conv4_bn(self.conv4(x))))
        x = x.view(-1, 128*12*12)
        x = F.dropout(F.relu(self.fc1(x)), training=self.training, p=0.4)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [12]:
model = Net(196)
model.to(device)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv1_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv4_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=18432, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=196, bias=True)
)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum=0.9)
from torch.optim.lr_scheduler import *
scheduler=StepLR(optimizer,step_size=3)

In [14]:
def train(model,device, train_loader, epoch):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        x,y= data
        x=x.to(device)
        y=y.to(device)
        optimizer.zero_grad()
        y_hat= model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
    print ('Train Epoch: {}\t Loss: {:.6f}'.format(epoch,loss.item()))

In [15]:
def valid(model, device, valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    with torch.no_grad():
        for i,data in enumerate(valid_loader):          
            x,y= data
            x=x.to(device)
            y=y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            valid_loss += criterion(y_hat, y).item() # sum up batch loss
            pred = y_hat.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
    valid_loss /= len(valid_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        valid_loss, correct, len(valid_dataset),
        100. * correct / len(valid_dataset)))

In [17]:
for epoch in range(1, 100):
    train(model=model, device=device, train_loader=train_loader, epoch=epoch)
    valid(model=model, device=device, valid_loader=valid_loader)

Train Epoch: 1	 Loss: 5.324077

Test set: Average loss: 0.0844, Accuracy: 9/1628 (1%)

Train Epoch: 2	 Loss: 5.313012

Test set: Average loss: 0.0844, Accuracy: 9/1628 (1%)

Train Epoch: 3	 Loss: 5.303848

Test set: Average loss: 0.0844, Accuracy: 9/1628 (1%)

Train Epoch: 4	 Loss: 5.291794

Test set: Average loss: 0.0844, Accuracy: 9/1628 (1%)

Train Epoch: 5	 Loss: 5.281834

Test set: Average loss: 0.0844, Accuracy: 9/1628 (1%)

Train Epoch: 6	 Loss: 5.273494

Test set: Average loss: 0.0844, Accuracy: 9/1628 (1%)

Train Epoch: 7	 Loss: 5.263770

Test set: Average loss: 0.0844, Accuracy: 9/1628 (1%)

Train Epoch: 8	 Loss: 5.256256

Test set: Average loss: 0.0845, Accuracy: 9/1628 (1%)

Train Epoch: 9	 Loss: 5.250169

Test set: Average loss: 0.0845, Accuracy: 9/1628 (1%)

Train Epoch: 10	 Loss: 5.238633

Test set: Average loss: 0.0845, Accuracy: 9/1628 (1%)

Train Epoch: 11	 Loss: 5.241565

Test set: Average loss: 0.0845, Accuracy: 12/1628 (1%)

Train Epoch: 12	 Loss: 5.226976

Test se

Train Epoch: 94	 Loss: 5.108614

Test set: Average loss: 0.0847, Accuracy: 11/1628 (1%)

Train Epoch: 95	 Loss: 5.092119

Test set: Average loss: 0.0846, Accuracy: 10/1628 (1%)

Train Epoch: 96	 Loss: 5.088664

Test set: Average loss: 0.0846, Accuracy: 9/1628 (1%)

Train Epoch: 97	 Loss: 5.089634

Test set: Average loss: 0.0846, Accuracy: 10/1628 (1%)

Train Epoch: 98	 Loss: 5.093712

Test set: Average loss: 0.0846, Accuracy: 11/1628 (1%)

Train Epoch: 99	 Loss: 5.095280

Test set: Average loss: 0.0847, Accuracy: 10/1628 (1%)

